In [3]:
import psycopg2 as psg
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

from urlextract import URLExtract
extractor = URLExtract()

/home/shikha/Envs/algo360/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ImportError: No module named request

In [2]:
final_sms_df = pd.read_csv('/mnt/c/Algo360/pythoncodes/BANK/Finaldata_pkl/BankTrainingData.csv',usecols=['message'])
final_sms_df['message'] = final_sms_df.message.astype(str)

NameError: name 'pd' is not defined

In [8]:
stopWords = stopwords.words('english')

def cleanData(sentence):
    processedList = ""
    sentence = sentence.lower()
    sentence = sentence.replace('a/c','account')
    sentence = sentence.replace('no.','no ')
    sentence = sentence.replace('rs.','rs ')
    sentence = re.sub(r',',' ',sentence)
    urls = extractor.find_urls(sentence)
    if urls:
        for url in urls:
            sentence = sentence.replace(url,'')
    sentence = re.sub(r'\d+[.]\d+',' ',sentence)
    sentence = re.sub(r'[^A-Za-z\s.]',r' ',sentence)
    sentence = re.sub(r'\n',r' ',sentence)
    sentence = re.sub(r'((x)\2{3,})','',sentence)
    sentence = ' '.join(sentence.split())
    return sentence

In [9]:
final_sms_df['clean_message'] = final_sms_df.message.map(lambda x: cleanData(x))

In [6]:
tmp_corpus = final_sms_df['clean_message'].map(lambda x: x.split('.'))

In [7]:
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 1951368/1951368 [02:23<00:00, 13593.62it/s]


In [8]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 5506157
Num of words - 55038181


In [ ]:
phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

In [152]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

In [ ]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 100
window_size = 3 # sentences weren't too long, so
epochs = 20
min_count = 3
workers = 4

# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

In [ ]:
model.save('~/Workspace/sms_w2v_model')